# 알고리즘 트레이딩 실습 1

<a href="https://colab.research.google.com/github/SLCFLAB/Fintech2022/blob/main/AT_Day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finance-datareader 에서 데이터 받아오기

Reference:    
https://financedata.github.io/posts/finance-data-reader-users-guide.html

In [ ]:
!pip install -U finance-datareader

In [ ]:
!pip install cvxopt

In [ ]:
import FinanceDataReader as fdr
print(fdr.__version__)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cvxopt as opt
from cvxopt import solvers
import warnings

# 경고 메세지를 띄우고 싶지 않다면 아래 주석 제거
#warnings.filterwarnings('ignore')

## StockListing 함수

거래소에 속한 종목을 가져오는 데 사용. Input: **market**

한국 종목
* KRX - 한국 전체 거래소(코스피 + 코스닥 + 코넥스)
* KOSPI, KOSDAQ, KONEX

미국 종목
* NASDAQ, NYSE, AMEX, SP500

In [ ]:
sp500_listing = fdr.StockListing('SP500')
sp500_listing

In [ ]:
kospi_listing = fdr.StockListing('KOSPI')
kospi_listing

In [ ]:
kosdaq_listing = fdr.StockListing('KOSDAQ')
kosdaq_listing

미국 종목을 받아올 때와 한국 종목을 받아올 때의 column이 다르다는 것 확인 가능

### To-do

Q. S&P 500 종목에서 Sector의 개수를 구하시오.

Q. S&P 500 종목의 Sector 중 해당 Sector에 속하는 종목의 수가 5번째로 많은 Sector를 구하시오.

Q. 코스피 종목 중 지역이 서울에 위치해 있으며 2번째로 최근에 상장된 종목의 종목명과 섹터를 구하시오.

## DataReader 함수

실질적인 증시 데이터를 불러오는 데에 사용할 수 있는 함수는 `DataReader` 함수. Input으로 Symbol은 반드시 들어가야 함. 해당 Symbol을 확인할 수 있는 함수가 `StockListing`이라고 볼 수 있다.

In [ ]:
fdr.DataReader('006840', '2019-01-01', '2019-12-31')

DataReader를 통해 받은 dataframe의 Column 정보 
* Open: 시가
* High: 고가
* Low: 저가
* Close: 종가
* Volume: 거래량
* Change: 등락폭

In [ ]:
plt.figure(figsize=(10, 10))

dt = fdr.DataReader('006840', '2011') #2011년~현재
plt.plot(dt['Close'])

In [ ]:
# KS11	KOSPI 지수
# KQ11	KOSDAQ 지수
# KS50	KOSPI 50 지수
# KS100	KOSPI 100
# KRX100	KRX 100
# KS200	코스피 200

# DJI	다우존스 지수
# IXIC	나스닥 지수
# US500	S&P 500 지수
# VIX	S&P 500 VIX

# JP225	닛케이 225 선물
# STOXX50E	Euro Stoxx 50
# CSI300	CSI 300 (중국)
# HSI	항셍 (홍콩)
# FTSE	영국 FTSE
# DAX	독일 DAX 30
# CAC	프랑스 CAC 40

# 환율 데이터: USD/KRX (원달러 환율), USD/EUR(달러당 유로화 환율), CNY/KRW: 위엔화 원화 환율


# 암호 화폐
# 암호화폐 가격: BTC/USD (비트코인 달러 가격, Bitfinex), BTC/KRW (비트코인 원화 가격, 빗썸)

In [ ]:
# KS11 (KOSPI 지수), 2015년~현재
df = fdr.DataReader('KS11', '2015')
plt.plot(df['Close'])
plt.show()

In [ ]:
# 원달러 환율, 1995년~현재
df = fdr.DataReader('USD/KRW', '1995')
plt.plot(df['Close'])
plt.show()

### To-do

Q. 삼성전자의 2019년 ~ 현재까지의 종가를 도식화하시오.

Q. 삼성전자의 등락폭이 가장 컸던 날의 날짜와 해당 날짜의 등락폭을 구하시오.

# 데이터 전처리 하기

In [ ]:
dt = fdr.StockListing('SP500')

In [ ]:
a = {}

for name in dt['Symbol'][:20]:
    print(name)
    try:
        a[name] = fdr.DataReader(name, '2018-12-31', '2019-12-31')['Close']
    except:
        print('except')
        pass

In [ ]:
a['MMM']

In [ ]:
k = pd.DataFrame(a)

In [ ]:
k.shape

In [ ]:
k

In [ ]:
plt.figure(figsize=(50, 50))
k.plot()

In [ ]:
kk = k / k.iloc[0]

In [ ]:
kk

In [ ]:
kk.plot()

# Expected Return and Covariance 구하기

In [ ]:
data = k
holding_periods=1

In [ ]:
returns = data.pct_change(holding_periods).iloc[holding_periods:] # holding period return vector

In [ ]:
returns

In [ ]:
data

In [ ]:
data.shift(holding_periods)

In [ ]:
returns2 = (data - data.shift(holding_periods)) / data.shift(holding_periods)

In [ ]:
log_returns = np.log(data) - np.log(data.shift(holding_periods))

In [ ]:
log_returns

In [ ]:
holding_periods=1
n_term=252 # dailiy return을 annualize
data = k

returns = data.pct_change(holding_periods).iloc[holding_periods:] # holding period return vector

exp_rets = returns.mean() * n_term # average return -- > APR(annual percentage rate)
cov_mat = returns.cov() * n_term # covariance
corr_mat = returns.corr() # corr

In [ ]:
corr_mat

In [ ]:
cov_mat = returns.cov()

In [ ]:
var = np.diag(cov_mat)

In [ ]:
annulized_var = var * n_term

In [ ]:
daily_std = returns.std()

In [ ]:
annulized_std = daily_std * np.sqrt(252)

In [ ]:
returns.cov()

# To-do

다음의 조건에 해당하는 각 지수별 price data를 pandas의 DataFrame 형태로 생성하시오.      
https://financedata.github.io/posts/finance-data-reader-users-guide.html
* 조건 1: 각 row의 인덱스는 날짜, 각 컬럼은 KOSPI200 지수, S&P500 지수, 나스닥 지수, 닛케이225 선물, USD/KRW로 구성.
* 조건 2: 종가 기준, 기간은 2010년 이후 현재까지.



 KS200:	코스피 200, US500:	S&P 500 지수, JP225:	닛케이 225 선물



In [ ]:
symbols = ['KS200', 'US500', 'IXIC', 'JP225', 'USD/KRW']

a = {}
for symbol in symbols:
    a[symbol] = fdr.DataReader(symbol, '2009-12-31')['Close']

In [ ]:
a = pd.DataFrame(a)

In [ ]:
a

위에서 생성된 DataFrame을 활용하여 다음의 데이터를 만드시오.    
a) daily return vector.    
b) (annulaize된) expected return    
c) covariance matrix    
d) correlation matrix

In [ ]:
holding_periods=1
n_term=252 # dailiy return을 annualize
data = a

In [ ]:
returns = data.pct_change(holding_periods).iloc[holding_periods:]

In [ ]:
exp_rets = returns.mean() * n_term # average return -- > APR(annual percentage rate)

In [ ]:
cov_mat = returns.cov() * n_term # covariance

In [ ]:
corr_mat = returns.corr() # corr

# Efficient Frontier 구하기

Reference: 
https://cvxopt.org/userguide/coneprog.html?highlight=qp#cvxoptsolvers.qp

In [ ]:
def mean_var_portfolio(cov_mat, exp_rets, target_ret):
    
    n = len(cov_mat)

    # Objective function
    # min (1/2)*w.T*P*w
    P = opt.matrix(cov_mat.values) 
    q = opt.matrix(0.0, (n, 1))

    # Constraints Gw <= h
    # exp_rets*w >= target_ret and w >= 0
    G = opt.matrix(np.vstack((-exp_rets.values, -np.identity(n))))
    h = opt.matrix(np.vstack((-target_ret, np.zeros((n, 1)))))

    # Constraints Aw = b
    # sum(w) = 1
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # Solve
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)

    # Put weights into a labeled series
    w = pd.Series(sol['x'], index=cov_mat.index)

    return w

In [ ]:
np.array(exp_rets)

In [ ]:
np.sqrt(np.diag(cov_mat))

In [ ]:
plt.matshow(corr_mat)
plt.show()

In [ ]:
corr_mat.style.background_gradient(cmap='coolwarm')

In [ ]:
import seaborn as sns
sns.heatmap(corr_mat)

In [ ]:
exp_rets

In [ ]:
np.sqrt(np.diag(cov_mat)).max()

In [ ]:
exp_rets.max()

In [ ]:
exp_rets[exp_rets > 0].min()

In [ ]:
max_risk = 0.5
max_ret = exp_rets.max()
min_ret = exp_rets[exp_rets > 0].min()

n_step = 100
step_diff = (max_ret - min_ret)/n_step

risks = []
rets = []

for step in range(n_step):
    target_ret = min_ret + (step * step_diff) # y축을 아래부터 시작해서 위로 조금씩 올려가면서
    _w = mean_var_portfolio(cov_mat, exp_rets, target_ret) # 최적화를 돌려서 나온 _w임.
    risk = np.sqrt(np.dot(np.dot(_w, cov_mat), _w))
    
    if risk > max_risk:
        break
    
    risks.append(risk)
    ret = np.dot(exp_rets, _w)
    rets.append(ret)

In [ ]:
risks

In [ ]:
rets

In [ ]:
plt.plot(risks, rets)
plt.show()

In [ ]:
plt.plot(risks, rets)
var = np.diag(cov_mat)
plt.scatter(np.sqrt(var), exp_rets)
plt.show()

In [ ]:
plt.plot(risks, rets)
var = np.diag(cov_mat)
plt.scatter(np.sqrt(var), exp_rets)
plt.show()

## To-Do

위 실습 1,2에서 구한 expected return과 covriance를 바탕으로 아래의 코드의 빈칸을 채워 Efficient frontier를 그려보시오

In [ ]:
max_risk=0.5
max_ret = ??
min_ret = ??

n_step = 1000
step_diff = (max_ret - min_ret)/n_step

risks = []
rets = []

for step in range(n_step):
    target_ret = min_ret + (step * step_diff) # y축을 아래부터 시작해서 위로 조금씩 올려가면서
    _w = ?? # 최적화를 돌려서 나온 _w임.
    risk = ??

    if risk > max_risk:
        break
    
    risks.append(risk)
    ret = ??
    rets.append(ret)

In [ ]:
plt.plot(risks, rets)
plt.show()